In [1]:
# MNIST Handwritten Digit Classification
import torch
import torch.nn as nn               # to load neural network functionality
import torch.nn.functional as nnF       # Adds efficiency
from torch.utils.data import DataLoader # Data loading in batches
from torchvision import datasets, transforms
import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix # For evaluating results
import matplotlib.pyplot as plt
%matplotlib inline
import time
torch.manual_seed(2002) # setting seed for consistent results
